In [ ]:
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import numpy as np

## Run

In [ ]:
path = 'logs/WSCC_9bus_coupled_0_0_0/'
logName = 'WSCC_9bus_coupled_0_0_0'
logFilename = path + logName
print(logFilename)

In [ ]:
%%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/build/dpsim/examples/cxx

WSCC_9bus_mult_coupled -U KLU ${TOP}/examples/CIM/WSCC-09_RX/*.xml

In [ ]:
%%capture
ts_coupled_sparse_rv = rt.read_timeseries_dpsim(logFilename+'_RightVector.csv')
ts_coupled_sparse_lv = rt.read_timeseries_dpsim(logFilename+'_LeftVector.csv')

In [ ]:
%%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/build/dpsim/examples/cxx

WSCC_9bus_mult_coupled -U SparseLU ${TOP}/examples/CIM/WSCC-09_RX/*.xml

In [ ]:
%%capture
ts_coupled_klu_rv = rt.read_timeseries_dpsim(logFilename+'_RightVector.csv')
ts_coupled_klu_lv = rt.read_timeseries_dpsim(logFilename+'_LeftVector.csv')

## Validate Results

In [ ]:
tolerance = 1e-4

for entry in ts_coupled_sparse_rv:
    sparse_values = ts_coupled_sparse_rv[entry].values
    klu_values = ts_coupled_klu_rv[entry].values
    # 2-norm of errors
    error = np.linalg.norm(sparse_values-klu_values, ord=2)
    assert(error < tolerance)

for entry in ts_coupled_sparse_lv:
    sparse_values = ts_coupled_sparse_lv[entry].values
    klu_values = ts_coupled_klu_lv[entry].values
    # 2-norm of errors
    error = np.linalg.norm(sparse_values-klu_values, ord=2)
    assert(error < tolerance)